In [2]:
import pandas as pd
import os
from glob import glob

In [12]:
!PYTHONPATH=.

In [15]:
RU_WEBS_DIR = "./pages-arch/pages-arch/ru"
EN_WEBS_DIR = "./pages-arch/pages-arch/en"

ru_webs_pl = glob(f"{RU_WEBS_DIR}/**.html")
en_webs_pl = glob(f"{EN_WEBS_DIR}/**.html")
print(f"Russian websites amount:{len(ru_webs_pl)}")
print(f"English websites amount:{len(en_webs_pl)}")

Russian websites amount:195
English websites amount:565


In [16]:
raw_ru = []
raw_en = []

for pl in ru_webs_pl:
    with open(pl, "r") as f:
        raw_ru.append(f.read())
for pl in en_webs_pl:
    with open(pl, "r") as f:
        raw_en.append(f.read())     

In [20]:
ru_df = pd.DataFrame({"raw": raw_ru})
ru_df["lang"] = "RU"
en_df = pd.DataFrame({"raw": raw_en})
en_df["lang"] = "EN"
df = pd.concat([ru_df, en_df])
df.head()

,raw,lang
0,"<!doctype html>\n<html data-n-head-ssr lang=""r...",RU
1,"<!doctype html><html class=""hcfe"" data-page-ty...",RU
2,"<!doctype html>\n<html data-n-head-ssr lang=""r...",RU
3,"<!doctype html>\n<html data-n-head-ssr lang=""r...",RU
4,"<!doctype html>\n<html data-n-head-ssr lang=""r...",RU


In [ ]:
#!pip install html2text

In [53]:
import html2text
h2t = html2text.HTML2Text()
h2t.ignore_links = True
df['text'] = df['raw'].apply(h2t.handle)

In [30]:
#!python -m spacy download en_core_web_sm
#!python -m spacy download ru_core_news_sm

/home/atheostheos/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-02-28 21:06:25.021400: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 21:06:25.058934: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 21:06:25.058970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 21:06:25.059809: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 21:06:25.065363: I tensorflow/core/platform/cpu_f

2024-02-28 21:06:38.399471: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-02-28 21:06:39.387633: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 4.8 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [54]:
import spacy
import spacy_transformers
nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")
en_df = df[df['lang'] == "EN"]
ru_df = df[df['lang'] == "RU"]

In [91]:
def get_tokens_en(text):
    doc = nlp_en(ex)
    return [w for w in doc if (w.is_alpha and not w.is_stop)]

def get_tokens_ru(text):
    doc = nlp_ru(ex)
    return [w for w in doc if (w.is_alpha and not w.is_stop)]

In [100]:
ts = get_tokens_en(en_df.iloc[0]["text"])

In [118]:
ts[69].lemma_

'image'

In [126]:
from tqdm.notebook import tqdm
token_set = set()
lemmas_set = dict()
for index, row in tqdm(df.iterrows()):
    lang = row['lang']
    doc = nlp_en(row['text']) if lang == "EN" else nlp_ru(row["text"])
    toks = [w for w in doc if (w.is_alpha and not w.is_stop)]
    token_set.update([t.lower_ for t in toks])
    for t in toks:
        lem = t.lemma_
        if lem in lemmas_set.keys():
            lemmas_set[lem].update([t.lower_])
        else:
            lemmas_set[lem] = set([t.lower_])
    

0it [00:00, ?it/s]

In [128]:
with open("tokens.txt", "w") as f:
    for word in token_set:
        f.write(word + "\n")

In [132]:
with open('lemmas.txt', "w") as f:
    for key, lem_set in lemmas_set.items():
        f.write(f"{key.lower()}: {' '.join(lem_set)}\n")